In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from keras.models import Sequential 
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

In [3]:
# 1. Importar la base de datos del circuito RC
RC_circuit_db = pd.read_csv('db_completa_RC_Tecnologia.csv')
RC_circuit_db.head()

,time,voltaje_ent,voltaje_cap_5,Resistencia real,Capacitancia Real
0,0,4.93,0.00,53300,100
1,200,4.93,0.00,53300,100
2,400,4.93,0.02,53300,100
3,600,4.93,0.24,53300,100
4,800,4.93,0.43,53300,100


In [3]:
# 2. Normalizar mis datos

In [4]:
def normalize(x, xmin, xmax):
    return (xmax - xmin)*((x - np.min(x))/(np.max(x) - np.min(x))) + xmin

In [5]:
#X = [tiempo, R, C, Ve]
#y = [Vc]

R = normalize(RC_circuit_db['Resistencia real'], 0.1, 0.8)
C = normalize(RC_circuit_db['Capacitancia Real'], 0.1, 0.8)
t = normalize(RC_circuit_db['time'], 0.1, 0.8)
Ve = RC_circuit_db['voltaje_ent']

# Input variables
X = np.vstack([R, C, t, Ve]).T

# Target
y = RC_circuit_db['voltaje_cap_5']

In [9]:
# 3. Segmentar la base de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 2002)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14000, 4)
(6000, 4)
(14000,)
(6000,)


In [13]:
# 4. Definimos la topologia de la ANN
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam


n_neurons = 16  # Aumentado el número de neuronas
model = Sequential(name="Improved-Model")
model.add(Input(shape=(X.shape[1],), name='Input-Layer'))
model.add(Dense(n_neurons, activation='relu', name='Hidden-Layer-1'))
model.add(Dropout(0.3))  # Dropout para regularización
model.add(Dense(n_neurons, activation='relu', name='Hidden-Layer-2'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear', name='Output-Layer'))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

Model: "Improved-Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Hidden-Layer-1 (Dense)          │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden-Layer-2 (Dense)          │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output-Layer (Dense)            │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 369 (1.44 KB)

 Trainable params: 369 (1.44 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# 5. Compile keras model
model.compile(optimizer = 'sgd', # default='rmsprop', an algorithm to be used in backpropagation
              loss = 'mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics = ['mean_squared_error','mean_absolute_error'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
            )

In [16]:
# 6. Entrenamos el modelo
history = model.fit(X_train, # input data
                    y_train, # target data
                    #batch_size = 10, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs = 500, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose = 1, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    #callbacks = [earlyStopping, mcp_save],
                    validation_data = (X_test, y_test))

Epoch 1/500
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 3.6407 - mean_absolute_error: 1.5966 - mean_squared_error: 3.6407 - val_loss: 1.9379 - val_mean_absolute_error: 1.1804 - val_mean_squared_error: 1.9379
Epoch 2/500
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.2159 - mean_absolute_error: 1.2431 - mean_squared_error: 2.2159 - val_loss: 1.1917 - val_mean_absolute_error: 0.8863 - val_mean_squared_error: 1.1917
Epoch 3/500
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.4844 - mean_absolute_error: 0.9915 - mean_squared_error: 1.4844 - val_loss: 1.1190 - val_mean_absolute_error: 0.8667 - val_mean_squared_error: 1.1190
Epoch 4/500
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.3098 - mean_absolute_error: 0.9214 - mean_squared_error: 1.3098 - val_loss: 1.0122 - val_mean_absolute_error: 0.8072 - val_mean_squared_error: 1.0122
Epoch 5/500
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.2777 - mean_absolute_error: 0.9020 - mean_squared_error: 1.2777 - val_loss: 1.5883 - 

In [18]:
def regression_results(y_true, y_pred):
    # Regression metrics
    ev = explained_variance_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred) 
    mse = mean_squared_error(y_true, y_pred) 
    r2 = r2_score(y_true, y_pred)

    print('explained_variance: ', round(ev,4))    
    print('MAE: ', round(mae,4))
    print('MSE: ', round(mse,4))
    print('R²: ', round(r2,4))

In [19]:
# 7. Evaluamos el modelo

pred_tr = model.predict(X_train)
pred_te = model.predict(X_test)

#%% Model Performance Summary    
print("")
print('---------- Evaluation on Training Data ----------')
regression_results(y_train, pred_tr)
print("")

print('---------- Evaluation on Test Data ----------')
regression_results(y_test, pred_te)
print("")

#%% Results per output
mse_train = mean_squared_error(y_train, pred_tr)

r2_train = r2_score(y_train, pred_tr)

mse_test = mean_squared_error(y_test, pred_te)

r2_test = r2_score(y_test, pred_te)

col_names = ('MSE (train)', 'R2 (train)', 'MSE (test)', 'R2 (test)')

df = np.array([mse_train, r2_train, mse_test, r2_test])

print("")
print('---------- Evaluation per output ----------')
results = pd.DataFrame(data = df.reshape(1,-1), columns = col_names)
print(results)

123/438 [=======>......................] - ETA: 0s

2024-11-13 11:35:53.442933: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - 0s 1ms/step

---------- Evaluation on Training Data ----------
explained_variance:  0.8962
MAE:  0.3722
MSE:  0.3129
R²:  0.8905

---------- Evaluation on Test Data ----------
explained_variance:  0.9002
MAE:  0.3629
MSE:  0.3055
R²:  0.8945


---------- Evaluation per output ----------
   MSE (train)  R2 (train)  MSE (test)  R2 (test)
0      0.31291    0.890485    0.305515   0.894514


In [ ]:
# Prueba de pendiente-intercepto

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Carga tus datos reales y predicciones del modelo ANN
# y_true = ... # Valores reales
# y_pred_ann = ... # Predicciones del modelo ANN

# Calcula el Error Cuadrático Medio (MSE)
mse_ann = mean_squared_error(y_true, y_pred_ann)

# Define la tolerancia para el P-I Test
tolerancia = 1.96 * np.sqrt(mse_ann)  # Intervalo de confianza al 95%

# Realiza el P-I Test
p_i_test_ann = np.abs(y_true - y_pred_ann) <= tolerancia

# Calcula la proporción de valores dentro del intervalo
proporcion_ann = np.mean(p_i_test_ann)

print(f"P-I Test Proporción ANN: {proporcion_ann}")


NameError: name 'y_true' is not defined